In [225]:
import numpy as np
import pandas as pd

class ANFIS:
    def __init__(self, params_layer1, params_layer4):
        self.layer1 = Layer1(params_layer1)
        self.layer2 = Layer2()
        self.layer3 = Layer3()
        self.layer4 = Layer4()
        self.layer5 = Layer5()
        self.params_layer4 = params_layer4

    def train(self, X, Y, epochs=100, learning_rate=0.01):
        for epoch in range(epochs):
            # Forward pass
            membership_A = self.layer1.forward(X[:, :15])  # Assuming first 20 features are used
            membership_B = self.layer1.forward(X[:, 15:])  # Assuming remaining 10 features are used
            weights = self.layer2.forward(membership_A, membership_B)
            normalized_weights = self.layer3.forward(weights)
            predictions = self.layer4.forward(normalized_weights, X[:, :15], X[:, 15:], self.params_layer4['p'], self.params_layer4['q'], self.params_layer4['r'])
            # Calculate loss
            loss = np.mean((predictions - Y) ** 2)

            # Backward pass (backpropagation can be implemented here)

            # Update parameters (parameter updates can be implemented here)

            # Print epoch and loss
            print(f"Epoch {epoch + 1}, Loss: {loss}")

    def predict(self, X):
        membership_A = self.layer1.forward(X[:, :15])  # Assuming first 15 features are used
        membership_B = self.layer1.forward(X[:, 15:])  # Assuming next 15 features are used


        weights = self.layer2.forward(membership_A, membership_B)
        normalized_weights = self.layer3.forward(weights)
        predictions = self.layer5.forward(normalized_weights, Y)
        
        return predictions

class Layer1:
    def __init__(self, params):
        self.a = params['a']
        self.b = params['b']
        self.c = params['c']

    def gbellmf(self, x, a, b, c):
        return 1 / (1 + np.abs((x - self.c) / self.a) ** (2 * self.b))

    def forward(self, x):
        num_samples, num_features = x.shape
        output = []
        for i in range(num_samples):  # Iterate over samples
            sample_output = []
            for j in range(num_features):  # Iterate over features
                fuzzified_value = self.gbellmf(x[i, j], self.a[j], self.b[j], self.c[j])
                sample_output.append(fuzzified_value)
            output.append(sample_output)
        return output


class Layer2:
    def forward(self, membership_A, membership_B):
        output = []
        for i in range(len(membership_A)):
            row = []
            for j in range(len(membership_A[i])):
                w_ij = membership_A[i][j] * membership_B[i][j]
                row.append(w_ij)
            output.append(row)
        return output

class Layer3:
    def forward(self, weights):
        # Flatten the list if the elements are lists
        if isinstance(weights[0], list):
            weights = [sum(sublist) for sublist in weights]
        total_weight = sum(weights)
        output = [w_i / total_weight for w_i in weights]
        return output

class Layer4:
    def forward(self, normalized_weights, x, y, p, q, r):
        num_samples, num_features = x.shape
        output = np.zeros((len(normalized_weights), num_samples))  # Output matrix initialization
        for i, w_bar_i in enumerate(normalized_weights):
            output[i] = w_bar_i * (np.sum(p.reshape(1, -1) * x, axis=1) + np.sum(q.reshape(1, -1) * y, axis=1) + r)
        return output


class Layer5:
    def forward(self, weights, f_values):
        sum_wf = sum([weights[i] * f_values[i] for i in range(len(weights))])
        sum_w = sum(weights)
        output = sum_wf / sum_w
        return output

# Load your dataset
data = pd.read_csv("bcd.csv")

X = data.iloc[:, :-1].values.astype(int)
Y = data.iloc[:, -1].values.astype(int)

num_features = 15
# Assuming you already have parameters for Layer 1 and Layer 4
params_layer1 = {
    'a': np.random.rand(num_features),  
    'b': np.random.rand(num_features),
    'c': np.random.rand(num_features)
}

params_layer4 = {'p': np.random.rand(15), 'q': np.random.rand(15), 'r': np.random.rand(15)}

anfis_model = ANFIS(params_layer1, params_layer4)


# Predict with trained model
predictions = anfis_model.predict(X)

print("Tahminler:", predictions)
print("Gerçek Değerler:", Y)



Tahminler: [0.34920584 0.36097589 0.34847528 0.35189733 0.35301492 0.3428387
 0.35580673 0.352548   0.34797977 0.37088476 0.35754878 0.35228488
 0.350172   0.35269848 0.35192353]
Gerçek Değerler: [1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0
 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0
 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1
 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1]
